In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder \
  .appName('Spotify Top Songs') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-3.3-bigquery:0.37.0') \
  .getOrCreate()

In [ ]:
df = spark.read \
        .format("bigquery") \
        .option("table", "dezoomcampproject.spotify.top_songs_partitoned_clustered") \
        .load()

In [ ]:
df.printSchema()


In [ ]:
df.show()

In [ ]:
#Convert the column 'snapshot_date' from datetime to date.
df= df.withColumn("snapshot_date", F.to_date(F.col("snapshot_date")))

In [ ]:
#Convert the column 'album_release_date' from datetime to date.
df= df.withColumn("album_release_date", F.to_date(F.col("album_release_date")))


In [ ]:
# Substitute null values in the 'country' column with 'Global'
df = df.withColumn("country", F.when(F.col("country").isNull(), "Global").otherwise(F.col("country")))

In [ ]:
global_df = df.filter(df["country"] == "Global")


In [ ]:
# Show the filtered DataFrame
global_df.show()

In [ ]:
 # Write transformed data back to BigQuery
df.write \
    .format("bigquery") \
    .option("temporaryGcsBucket", "dezoomcampproject-terra-bucket") \
    .option("table", "dezoomcampproject.spotify.top_songs_partitoned_clustered") \
    .mode("overwrite") \
    .save()